In [ ]:
!pip -q install pandas

import json, re, math, random
from collections import defaultdict, Counter
from unicodedata import normalize
import pandas as pd

In [ ]:
# ====== 0) 데이터 로드 ======
# Colab 기준: 본인 경로로 바꾸세요.
DATA_PATH = "/content/drive/MyDrive/sejong_dataset.json"   # 예: Google Drive 마운트 시 경로
with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
# ====== 1) 유형 태깅 ======
# - 규칙 기반으로 5유형 분류
# - 필요 시 규칙을 추가/완화하세요.
ROLE_PAT = r"(너는\s*세종이다|전하|폐하|상감|왕의\s*이름|세종의\s*이름|외국인이|백성이)"
DOC_PAT  = r"(\[자료\]|다음\s*자료|본문|발췌|근거를\s*바탕)"
REASON_PAT = r"(분석하라|설명하라|의의|의미|특징을\s*분석|평가하라|해석하라|교훈|정리하라|시사점)"
FACT_Q_PAT = r"(언제|누구|무엇|어디|몇\s*대|몇\s*년|무엇이었는가|무엇인가|어떤\s*가|언제인가|\?)"
REFUSAL_PAT = r"(논할\s*처지|알지\s*못하노라|모르노라|할\s*수\s*없|그\s*분야의\s*전문가|후세의|처지가\s*아니)"

def tag_type(ex):
    instr = (ex.get("instruction", "") or "").strip()
    inp   = (ex.get("input", "") or "").strip()
    out   = (ex.get("output", "") or "").strip()

    # 1) 경계 설정/거부: 출력에서 거부 신호가 뚜렷하면 최우선
    if re.search(REFUSAL_PAT, out):
        return "경계 설정 및 응답 거부"

    # 2) 자료 기반: input이나 instruction에 자료/근거 지시
    if re.search(DOC_PAT, inp) or re.search(DOC_PAT, instr):
        return "자료 기반 요약/추출"

    # 3) 역할 수행: 역할/상황 연기 지시
    if re.search(ROLE_PAT, instr) or re.search(ROLE_PAT, inp):
        return "역할 수행 대화"

    # 4) 심층 추론/철학: 분석/설명/의의/해석 등
    if re.search(REASON_PAT, instr):
        return "심층 추론 및 철학"

    # 5) 사실기반 Q&A: 의문사·사실 질의 형태
    if re.search(FACT_Q_PAT, instr) and not inp:
        return "사실기반 질의응답"

    # 기본값(못 맞춘 경우): 사실기반으로 보되 필요 시 수동 검토
    return "사실기반 질의응답"

labels = [tag_type(ex) for ex in data]

# 태그를 항목에 추가(선택)
for ex, lab in zip(data, labels):
    ex["task_type"] = lab

In [ ]:
# ====== 2) 전체 분포 확인 ======
def dist(labels):
    c = Counter(labels)
    total = sum(c.values())
    rows = [{"type": k, "count": v, "ratio": round(v/total, 4)} for k, v in sorted(c.items())]
    return pd.DataFrame(rows)

overall_df = dist(labels)
print("Overall distribution:")
print(overall_df)

Overall distribution:
            type  count   ratio
0  경계 설정 및 응답 거부     41  0.0147
1      사실기반 질의응답   1464  0.5266
2     심층 추론 및 철학    907  0.3263
3       역할 수행 대화    244  0.0878
4    자료 기반 요약/추출    124  0.0446


In [ ]:
# ====== 3) 유형별 검증셋 할당량 계산 (라지스트 리메인더) ======
VAL_RATIO = 0.05
N = len(data)
target_val_total = round(N * VAL_RATIO)

idx_by_type = defaultdict(list)
for i, lab in enumerate(labels):
    idx_by_type[lab].append(i)

# 우선 바닥(floor) 할당 + 최소/최대 제약 적용
base = {}
rem = {}
for lab, idxs in idx_by_type.items():
    quota = len(idxs) * VAL_RATIO
    b = math.floor(quota)
    # 한 클래스 샘플이 1개면 검증으로 못 보냄(학습에 0개 남는 것 방지)
    if len(idxs) == 1:
        b = 0
    else:
        b = max(1, min(len(idxs) - 1, b))
    base[lab] = b
    rem[lab] = quota - math.floor(quota)

# 총합 맞추기(라지스트 리메인더 방식으로 +1/-1 조정)
val_total = sum(base.values())
need = target_val_total - val_total

if need > 0:
    # 잔여가 큰 유형부터 1씩 추가(여유가 있을 때만)
    for lab in sorted(rem, key=rem.get, reverse=True):
        if need == 0: break
        if base[lab] < len(idx_by_type[lab]) - 1:
            base[lab] += 1
            need -= 1
elif need < 0:
    # 잔여가 작은 유형부터 1씩 감축(0 미만 금지)
    for lab in sorted(rem, key=rem.get):
        if need == 0: break
        if base[lab] > 0:
            base[lab] -= 1
            need += 1

In [ ]:
# ====== 4) 검증 세트 샘플링 및 분할 ======
random.seed(2025)
val_idx = set()
for lab, idxs in idx_by_type.items():
    k = min(base[lab], len(idxs))
    if k > 0:
        val_idx.update(random.sample(idxs, k))

train_idx = [i for i in range(N) if i not in val_idx]

train = [data[i] for i in train_idx]
val = [data[i] for i in val_idx]

In [ ]:

# ====== 5) 분포 검증 ======
train_df = dist([ex["task_type"] for ex in train])
val_df = dist([ex["task_type"] for ex in val])

print("\nTrain distribution:")
print(train_df)
print("\nVal distribution:")
print(val_df)


Train distribution:
            type  count   ratio
0  경계 설정 및 응답 거부     39  0.0148
1      사실기반 질의응답   1391  0.5267
2     심층 추론 및 철학    861  0.3260
3       역할 수행 대화    232  0.0878
4    자료 기반 요약/추출    118  0.0447

Val distribution:
            type  count   ratio
0  경계 설정 및 응답 거부      2  0.0144
1      사실기반 질의응답     73  0.5252
2     심층 추론 및 철학     46  0.3309
3       역할 수행 대화     12  0.0863
4    자료 기반 요약/추출      6  0.0432


In [ ]:
# ====== 6) 저장(JSONL 권장) ======
def save_jsonl(path, items):
    with open(path, "w", encoding="utf-8") as f:
        for ex in items:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")

save_jsonl("/content/sejong_train_95.jsonl", train)
save_jsonl("/content/sejong_val_5.jsonl", val)

print(f"\nSaved: {len(train)} train -> /content/sejong_train_95.jsonl")
print(f"Saved: {len(val)}   val  -> /content/sejong_val_5.jsonl")


Saved: 2641 train -> /content/sejong_train_95.jsonl
Saved: 139   val  -> /content/sejong_val_5.jsonl


In [ ]:
# ====== 7) 드라이브에 저장 ======
# Colab 기준: 본인 경로로 바꾸세요.
DRIVE_SAVE_PATH = "/content/drive/MyDrive/" # 예: Google Drive 마운트 시 경로

!cp /content/sejong_train_95.jsonl {DRIVE_SAVE_PATH}
!cp /content/sejong_val_5.jsonl {DRIVE_SAVE_PATH}

print(f"\nCopied /content/sejong_train_95.jsonl to {DRIVE_SAVE_PATH}")
print(f"Copied /content/sejong_val_5.jsonl to {DRIVE_SAVE_PATH}")


Copied /content/sejong_train_95.jsonl to /content/drive/MyDrive/
Copied /content/sejong_val_5.jsonl to /content/drive/MyDrive/
